# Restaurant Visitor Forecasting by GooseLearning

In [1]:
import pandas as pd

In [2]:
data = {
    'air_store': pd.read_csv('dataset/air_store_info.csv'),
    'hpg_store': pd.read_csv('dataset/hpg_store_info.csv'),

    'air_reserve': pd.read_csv('dataset/air_reserve.csv'),
    'hpg_reserve': pd.read_csv('dataset/hpg_reserve.csv'),

    'air_hpd': pd.read_csv('dataset/store_id_relation.csv'),
    
    'holidays': pd.read_csv('dataset/date_info.csv').rename(columns={
        'calendar_date': 'visit_date',
        'holiday_flg'  : 'holiday',
    })[['visit_date', 'holiday']],

    'train': pd.read_csv('dataset/air_visit_data.csv'),
    'test':  pd.read_csv('dataset/sample_submission.csv'),
}

### Обработка данных

Приводим `test` к формату `train`

In [3]:
data['test']['air_store_id'] = [id[:20] for id in data['test']['id']]
data['test']['visit_date']   = [id[21:] for id in data['test']['id']]

Дополнительные колонки даты в `train` и `test`

In [4]:
for ds in ['train', 'test']:
    data[ds]['visit_date']  = pd.to_datetime(data[ds]['visit_date'])
    data[ds]['visit_day']   = data[ds]['visit_date'].dt.day
    data[ds]['visit_month'] = data[ds]['visit_date'].dt.month
    data[ds]['visit_year']  = data[ds]['visit_date'].dt.year
    data[ds]['visit_dow']   = data[ds]['visit_date'].dt.dayofweek
    data[ds]['visit_date']  = data[ds]['visit_date'].dt.strftime('%Y-%m-%d')
    
    data[ds] = pd.merge(data[ds], data['holidays'], how='left', on=['visit_date'])

In [5]:
data['test'].head()

,id,visitors,air_store_id,visit_date,visit_day,visit_month,visit_year,visit_dow,holiday
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,23,4,2017,6,0
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,24,4,2017,0,0
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,25,4,2017,1,0
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,26,4,2017,2,0
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,27,4,2017,3,0


Получаем набор доступных ID

In [6]:
data['air_store_ids'] = pd.DataFrame({
    'air_store_id': data['test']['air_store_id'].unique()
})

In [7]:
print('air_store_ids', len(data['air_store_ids']))

air_store_ids 821


 Расширяем `air_reserve` данными из `hpg_reserve`

In [8]:
data['hpg_reserve_air'] = pd.merge(data['hpg_reserve'], data['air_hpd'], how='inner', on=['hpg_store_id'])
data['air_reserve_ext'] = pd.concat([data['air_reserve'], data['hpg_reserve_air'].drop('hpg_store_id', axis=1)])

In [9]:
print('air_reserve    ', len(data['air_reserve']))
print('air_reserve_ext', len(data['air_reserve_ext']))

air_reserve     92378
air_reserve_ext 120561


Дополнительные колонки даты в `air_reserve_ext`

In [10]:
for dt_type in ['visit', 'reserve']:
    data['air_reserve_ext'][dt_type + '_datetime'] = pd.to_datetime(data['air_reserve_ext'][dt_type + '_datetime'])

data['air_reserve_ext']['visit_date'] = data['air_reserve_ext']['visit_datetime'].dt.strftime('%Y-%m-%d')
    
data['air_reserve_ext']['reserve_diff'] = data['air_reserve_ext']['visit_datetime'] - data['air_reserve_ext']['reserve_datetime']
data['air_reserve_ext']['reserve_diff'] = data['air_reserve_ext'].apply(lambda row: row['reserve_diff'].days, axis=1)

In [11]:
data['air_reserve_ext'].head()

,air_store_id,reserve_datetime,reserve_visitors,visit_datetime,visit_date,reserve_diff
0,air_877f79706adbfb06,2016-01-01 16:00:00,1,2016-01-01 19:00:00,2016-01-01,0
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,3,2016-01-01 19:00:00,2016-01-01,0
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,6,2016-01-01 19:00:00,2016-01-01,0
3,air_877f79706adbfb06,2016-01-01 16:00:00,2,2016-01-01 20:00:00,2016-01-01,0
4,air_db80363d35f10926,2016-01-01 01:00:00,5,2016-01-01 20:00:00,2016-01-01,0


Объединяем записи в `air_reserve_ext`

In [12]:
tmp = None
for op_type in ['sum', 'mean', 'median', 'min', 'max', 'count']:
    tmp2 = getattr(
        data['air_reserve_ext']
            .groupby(['air_store_id', 'visit_date'], as_index=False)
            [['reserve_diff', 'reserve_visitors']],
        op_type,
    )()
    
    tmp2 = tmp2.rename(columns={
        'reserve_diff':     'reserve_diff_'     + op_type,
        'reserve_visitors': 'reserve_visitors_' + op_type,
    })
    
    if tmp is None:
        tmp = tmp2
    else:
        tmp = pd.merge(tmp, tmp2, how='inner', on=['air_store_id', 'visit_date'])
    
data['air_reserve_ext'] = tmp

In [13]:
print('air_reserve_ext', len(data['air_reserve_ext']))
data['air_reserve_ext'].head()

air_reserve_ext 42193


,air_store_id,visit_date,reserve_diff_sum,reserve_visitors_sum,reserve_diff_mean,reserve_visitors_mean,reserve_diff_median,reserve_visitors_median,reserve_diff_min,reserve_visitors_min,reserve_diff_max,reserve_visitors_max,reserve_diff_count,reserve_visitors_count
0,air_00a91d42b08b08d9,2016-01-14,3,2,3.0,2.0,3.0,2.0,3,2,3,2,1,1
1,air_00a91d42b08b08d9,2016-01-15,5,4,5.0,4.0,5.0,4.0,5,4,5,4,1,1
2,air_00a91d42b08b08d9,2016-01-16,2,2,2.0,2.0,2.0,2.0,2,2,2,2,1,1
3,air_00a91d42b08b08d9,2016-01-22,3,2,3.0,2.0,3.0,2.0,3,2,3,2,1,1
4,air_00a91d42b08b08d9,2016-01-29,6,5,6.0,5.0,6.0,5.0,6,5,6,5,1,1


Добавляем данные в `train` и `test` 

In [14]:
for ds in ['train', 'test']:
    data[ds] = pd.merge(data[ds], data['air_reserve_ext'], how='left', on=['air_store_id', 'visit_date'])